# Install packages

In [ ]:
# devtools::install_version("mixsqp", version = "0.1-97", repos = "http://cran.us.r-project.org")
# devtools::install_version("ashr", version = "2.2-32", repos = "http://cran.us.r-project.org")
# devtools::install_github("jean997/cause@v1.0.0")
# for the latest version, use devtools::install_github("jean997/cause@v1.2.0")

# Run CAUSE

In [1]:
library(readr)
library(cause)
ts1 = c("AD", "ASD", "Daytime_Sleepiness", "Height_UKB",  "Intelligence", "RA",      
        "T2D", "Alcohol", "BMI", "Depression", "IBD", "MDD", "SCZ", "Angina", 
        "CAD", "HBP", "Income", "NEB", "Smoking", "Urate", "Anorexia", 
        "CD", "Height_GIANT", "Insomnia", "Neuroticism", "SWB")

ts2 = c("Hair_Light_Brown", "Hair_Dark_Brown",  "Hair_Black",    "Hair_Blonde",  "Tanning")

Threshold=1e-03

for( exposure in ts1 ){
  
  for( outcome in ts2 ){
    
    start = proc.time()
    
    # read GWAS summary statistics
    cat(exposure,"~",outcome,"\n")
    
    X1 = readr::read_delim(paste0("./GWAS_26and5_formatted/", exposure), delim="\t", escape_double = FALSE, trim_ws = TRUE, progress = F)
    
    X2 = readr::read_delim(paste0("./GWAS_26and5_formatted/", outcome), delim="\t", escape_double = FALSE, trim_ws = TRUE, progress = F)
    
    X1$b = X1$Z/sqrt(X1$N)
    X2$b = X2$Z/sqrt(X2$N)
    X1$se = 1/sqrt(X1$N)
    X2$se = 1/sqrt(X2$N)
    
    X <- try(gwas_merge(X1, X2, 
                        snp_name_cols = c("SNP", "SNP"),
                        beta_hat_cols = c("b", "b"),
                        se_cols = c("se", "se"),
                        A1_cols = c("A1", "A1"),
                        A2_cols = c("A2", "A2")))
    
    if(inherits(X , 'try-error')) next
    
    d0 = X1[, c("SNP", "P")]
    colnames(d0) = c("snp", "pval.exp")
    X0 = merge(X, d0, by="snp")
    
    # clump
    clumped = MRAPSS::clump(X0,
                            IV.Threshold = 1e-03,
                            SNP_col = "snp",
                            pval_col = "pval.exp",
                            clump_kb = 1000,
                            clump_r2 = 0.1,
                            bfile = "/import/home/share/xhu/database/1KG/all_1000G_EUR_Phase3",
                            plink_bin = "/import/home/maxhu/plink/plink")
    
    varlist <- with(X, sample(snp, size=min(nrow(X), 1000000), replace=FALSE))
    params <- try(est_cause_params(X, varlist))
    if(inherits(params , 'try-error')) next
    
    # cause
    if(!is.null(clumped)){
      
      top_ldl_pruned_vars =intersect(as.character(X$snp), as.character(subset(clumped, pval.exp <= Threshold)$snp))
      
      cause_res <- try(cause(X=X, variants = top_ldl_pruned_vars , param_ests = params, force=TRUE))
      
      if(inherits( cause_res , 'try-error')) next
      
      res_elpd <- data.frame(exposure,
                             outcome,
                             Threshold,
                             length(top_ldl_pruned_vars),
                             cause_res$elpd)
      
      res.cause.est = summary(cause_res, ci_size=0.95)
      
      res = data.frame(exposure, outcome,
                       Threshold,length(top_ldl_pruned_vars),
                       matrix(c(res.cause.est$quants[[2]][,1],
                                res.cause.est$quants[[2]][,2],
                                res.cause.est$quants[[2]][,3]), nrow=1))
      
      write.table(res, file="NC_CAUSE_est", append=T,
                  col.names = F, row.names = F,
                  quote = F)
      
      write.table(res_elpd, file="NC_CAUSE_elpd", append=T,
                  col.names = F, row.names = F,
                  quote = F)
      
      rm(top_ldl_pruned_vars)
      rm(res)
      rm(res_elpd)
      rm(res.cause.est)
      rm(cause_res)
      print(proc.time()-start)
      
    }
    
  }
  
}

AD ~ Hair_Light_Brown 


Rows: 1148476 Columns: 7

── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: "\t"
chr (3): SNP, A1, A2
dbl (4): Z, N, chi2, P


ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.

Rows: 1037742 Columns: 7

── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: "\t"
chr (3): SNP, A1, A2
dbl (4): Z, N, chi2, P


ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.



Formatting X1
There are  1148476  variants.
Removing  0  duplicated variants leaving  1148476 variants.
No variants have illegal alleles.
Removed  0  variants with ambiguous strand.
Flipping strand and effect allele so A1 is always A
Returning  1148476  variants.
Formatting X2
There are  1037742  variants.
Removing  0  duplicated variants leaving  1037742 variants.
No variants have illegal alleles.
Removed  0  variants with ambiguous strand.
Flipping strand and effect allele so A1 is always A
Returning  1037742  variants.
After merging and removing variants with inconsistent alleles,  there are  1015668  variants that are present in both studies and can be used with CAUSE.


API: public: http://gwas-api.mrcieu.ac.uk/

Clumping dgH2Hg, 2654 variants, using EUR population reference

Removing 1998 of 2654 variants due to LD with other variants or absence from LD reference panel



Estimating CAUSE parameters with  1000000  variants.
1 0.01038613 
2 5.12393e-07 
3 2.605879e-09 
Estimating CAUSE posteriors using  656  variants.


...

Formatting X1
There are  975109  variants.
Removing  0  duplicated variants leaving  975109 variants.
No variants have illegal alleles.
Removed  0  variants with ambiguous strand.
Flipping strand and effect allele so A1 is always A
Returning  975109  variants.
Formatting X2
There are  1037026  variants.
Removing  0  duplicated variants leaving  1037026 variants.
No variants have illegal alleles.
Removed  0  variants with ambiguous strand.
Flipping strand and effect allele so A1 is always A
Returning  1037026  variants.
After merging and removing variants with inconsistent alleles,  there are  926444  variants that are present in both studies and can be used with CAUSE.


Clumping wJoL6Q, 2581 variants, using EUR population reference

Removing 1898 of 2581 variants due to LD with other variants or absence from LD reference panel



Estimating CAUSE parameters with  926444  variants.
1 0.001370823 
2 1.615e-06 
3 1.905087e-09 
Estimating CAUSE posteriors using  683  variants.


Warning message:
“'compare' is deprecated.
Use 'loo_compare' instead.
See help("Deprecated")”


    user   system  elapsed 
1524.440 1361.159  327.152 


In [2]:
cause_elpd = unique(read.table("NC_CAUSE_elpd", header = F))
cause_est = unique(read.table("NC_CAUSE_est", header = F))
colnames(cause_elpd) = c("exposure","outcome","Threshold","nsnp","model1","model2","delta_elpd", "se_delta_elpd", "Z")
colnames(cause_est) = c("exposure","outcome","Threshold","nsnp", "b","b_l","b_u","eta","eta_l","eta_u","q","q_l","q_u")
cause_elpd = unique(subset(cause_elpd, model1=="sharing"&model2=="causal"))
cause_elpd$pval = pnorm(cause_elpd$Z)
cause_est = unique(cause_est[, c("exposure","outcome","Threshold","nsnp", "b","b_l","b_u")])
cause_est$se = (cause_est$b_u - cause_est$b_l)/2/1.96
cause_res = unique(merge(unique(cause_elpd[, c("exposure","outcome","pval")]),
                         cause_est[, c("exposure","outcome","Threshold","nsnp", "b","se")],
                         by=c("exposure","outcome")))
cause_res$Method = "CAUSE"
write.table(cause_res, file="NC_CAUSE_MRres", append=F, col.names = T, row.names = F, quote = F)